# CNN - MNIST

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print('TensorFlow Version: {}'.format(tf.__version__))

Found GPU at: /device:GPU:0
TensorFlow Version: 1.14.0-rc1


In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

Using TensorFlow backend.


## Downloading dataset

In [0]:
def load_mnist_data(dim):
    num_classes = 10
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    print(len(X_train), 'train samples')
    print(len(X_test), 'test samples')
    
    if dim == 3:
        if K.image_data_format() == 'channels_first':
            X_train = X_train.reshape(len(X_train), 1, 28, 28)
            X_test = X_test.reshape(len(X_test), 1, 28, 28)
        else:
            X_train = X_train.reshape(len(X_train), 28, 28, 1)
            X_test = X_test.reshape(len(X_test), 28, 28, 1)
    elif dim == 1:
        X_train = X_train.reshape(len(X_train), -1)
        X_test = X_test.reshape(len(X_test), -1)

    X_train = X_train.astype('float32') / 255.
    X_test = X_test.astype('float32') / 255.
    print('X_train shape:', X_train.shape)
    print('X_test shape:', X_test.shape)

    # convert class vectors to binary class matrices
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)
    return (X_train, y_train), (X_test, y_test)

In [4]:
img_rows, img_cols = 28, 28
num_classes = 10

(X_train, y_train), (X_test, y_test) = load_mnist_data(dim=3)

11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


## Main

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                 input_shape=X_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

print(model.summary())

W0624 04:53:39.030600 140266563553152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0624 04:53:39.047304 140266563553152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0624 04:53:39.057003 140266563553152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0624 04:53:39.110569 140266563553152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0624 04:53:39.115385 140266563553152 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [6]:
batch_size = 128
epochs = 10

model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_split=0.75)

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0624 04:53:43.563145 140266563553152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 15000 samples, validate on 45000 samples
Epoch 1/10
15000/15000 [==============================] - 9s 604us/step - loss: 0.5286 - acc: 0.8343 - val_loss: 0.1568 - val_acc: 0.9550
Epoch 2/10
15000/15000 [==============================] - 4s 270us/step - loss: 0.1787 - acc: 0.9462 - val_loss: 0.1039 - val_acc: 0.9683
Epoch 3/10
15000/15000 [==============================] - 4s 269us/step - loss: 0.1210 - acc: 0.9652 - val_loss: 0.0772 - val_acc: 0.9769
Epoch 4/10
15000/15000 [==============================] - 4s 269us/step - loss: 0.0948 - acc: 0.9725 - val_loss: 0.0684 - val_acc: 0.9795
Epoch 5/10
15000/15000 [==============================] - 4s 270us/step - loss: 0.0796 - acc: 0.9753 - val_loss: 0.0663 - val_acc: 0.9806
Epoch 6/10
15000/15000 [==============================] - 4s 270us/step - loss: 0.0716 - acc: 0.9773 - val_loss: 0.0653 - val_acc: 0.9807
Epoch 7/10
15000/15000 [==============================] - 4s 268us/step - loss: 0.0562 - acc: 0.9820 - val_loss: 0.0657 - 